# 自动并行计算

In [1]:
import mxnet as mx
from mxnet import gluon,nd

C:\Users\lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import sys
sys.path.append('../')

In [3]:
import gluonbook as gb
def run(x):
    return [nd.dot(x,x) for _ in range(10)]

In [4]:
x_cpu = nd.random.uniform(shape=(2000, 2000))
x_gpu = nd.random.uniform(shape=(6000, 6000), ctx=mx.gpu(0))

In [5]:
run(x_cpu) # 预热开始。
run(x_gpu)
nd.waitall() # 预热结束。

with gb.Benchmark('Run on CPU.'):
    run(x_cpu)
    nd.waitall()
with gb.Benchmark('Then run on GPU.'):
    run(x_gpu)
    nd.waitall()

Run on CPU. time: 2.7736 sec
Then run on GPU. time: 2.0456 sec


In [6]:
with gb.Benchmark('Run on both CPU and GPU in parallel.'):
    run(x_cpu)
    run(x_gpu)
    nd.waitall()

Run on both CPU and GPU in parallel. time: 3.3590 sec


In [7]:
def copy_to_cpu(x):
    return [y.copyto(mx.cpu()) for y in x]

In [8]:
with gb.Benchmark('Run on GPU.'):
    y = run(x_gpu)
    nd.waitall()
with gb.Benchmark('Then copy to CPU.'):
    copy_to_cpu(y)
    nd.waitall()

Run on GPU. time: 1.6725 sec
Then copy to CPU. time: 0.6877 sec


In [9]:
with gb.Benchmark('Run and copy in parallel.'):
    y = run(x_gpu)
    copy_to_cpu(y)
    nd.waitall()

Run and copy in parallel. time: 0.0020 sec
